# Implémentation 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras import layers
import tensorflow as tf
import keras 
from scipy.stats import norm

2022-04-15 10:57:14.689606: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-15 10:57:14.689632: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
class VAE(keras.Model):


    def __init__(self, encoder=None, decoder=None, loss_weights=[1,1], **kwargs):
  
        super(VAE, self).__init__(**kwargs)
        self.encoder      = encoder
        self.decoder      = decoder
        self.loss_weights = loss_weights
        print('loss_weights={list(self.loss_weights)}')
       
        
    def call(self, inputs):

        z_mean, z_log_var, z = self.encoder(inputs)
        output               = self.decoder(z)
        return output
                
        
    def train_step(self, input):

        if isinstance(input, tuple):
            input = input[0]
        
        k1,k2 = self.loss_weights
     
        with tf.GradientTape() as tape:
            
          
            z_mean, z_log_var, z = self.encoder(input)
            
           
            reconstruction       = self.decoder(z)
         
          
            reconstruction_loss  = k1 * tf.reduce_mean( keras.losses.binary_crossentropy(input, reconstruction) )

            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = -tf.reduce_mean(kl_loss) * k2

            total_loss = reconstruction_loss + kl_loss

    
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        
        return {
            "loss":     total_loss,
            "r_loss":   reconstruction_loss,
            "kl_loss":  kl_loss,
        }


# implémentation avec la construction personnelle de la couche LossLayer

In [4]:
class LossLayer(keras.layers.Layer):
   
    def __init__(self, loss_weights=[3,7]):
        super().__init__()
        self.k1 = loss_weights[0]
        self.k2 = loss_weights[1]


    def call(self, inputs):
        
     
        x, z_mean, z_log_var, y = inputs
        
      
        r_loss  = tf.reduce_mean( keras.losses.binary_crossentropy(x,y) ) * self.k1
      
        kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
        kl_loss = -tf.reduce_mean(kl_loss) * self.k2
        

        loss = r_loss + kl_loss
        self.add_loss(loss)
     
        self.add_metric(loss,   aggregation='mean',name='loss')
        self.add_metric(r_loss, aggregation='mean',name='r_loss')
        self.add_metric(kl_loss,aggregation='mean',name='kl_loss')
        
        return y

In [ ]:
inputs = keras.Input(shape=(28, 28, 1))
loss_weights  = [1,.001]

z_mean, z_log_var, z = encoder(inputs)
outputs              = decoder(z)

outputs = LossLayer(loss_weights=loss_weights)([inputs, z_mean, z_log_var, outputs])

vae=keras.Model(inputs,outputs)

vae.compile(optimizer='adam', loss=None)

In [ ]:
# entrainement 
vae.fit(x_data, epochs=epochs, batch_size=batch_size, shuffle=True)